# Project 2 - The Big Heist
## Group 2: Samer Alsaadi, Alexandre Beaubien, Mehmet Yazkanoglu
### Stats:
- Banks Robbed: 118
- Total Robbed: 6437500
- Time left: 0.1693

In [ ]:
# Data Imports

# Data Analysis
import numpy as np
import pandas as pd
import math

# Graph
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# Helper Functions
# Distance Function
def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# Functions to calculate time needed between banks based on 30kM/H limit
def time_taken(km):
    return (km/30) * 60

# Calculate distance between banks
def distance_between_banks(row):
    index = heist.index.get_loc(row.name)
    if index == 0:
        return 0
    prev_row = heist.iloc[index - 1]
    return distance(prev_row['x_coordinate'], prev_row['y_coordinate'], row['x_coordinate'], row['y_coordinate'])

In [ ]:
# Assign Dataframe
heist = pd.read_csv(r'C:\Users\samra\Desktop\Data Science Bootcamp\ds-project-the-big-heist\data\bank_data.csv')

## Algorithm Explaination
First we decided we had to manipulate the dataframe to create new columns. We determined that a few values would be important to add. The first one is a 'rate of return' that basically quantifies how much money we could get for the amount of time needed to rob the bank. We decided that this would be the driving factor in our "Greedy Algorithm" and we would sort the banks based on this new value. Once sorted by rate of return, the next value we needed to calculate was the distance between each bank and the previous bank starting with index 1. Then we figured we also needed to calculate the distance of each bank from the helipad (0,0).

Once we had this new dataframe, we created the loop. The loop was designed so that for each bank, it would calculate if the time needed to get to the next bank, rob it and return to the helipad was larger than the initialized time remaining. Because we decided to convert all of the time data to minutes, we initialized the time remaining to 1440 (24 hours). After verifying that there was enough time left to rob the bank and get back to the helipad (if necessary), we appended the bank ID to a list and substracted the amount of time it took to get there and rob it.

In [ ]:
'''
We initially have tried the algorithm below in which we take in the DataFrame and create variables that would be passed onto the algorithm.
We got a total of 5.9M with 1.6 Mins left, but as we enter the output list into the checker, it shows that we have two hours left.
Our testing showed that there might be an issue with the way it gets sorted by (rate_per_minute) as per Greedy method.
Actual solution is in the following cell.
'''

def heist_algorithm(heist):
    
    # Manipulate dataframe
    # Dollar per hour
    dollar_per_hour = heist['money']/heist['time (hr)']
    # Rate per minute
    rate_per_minute = round(dollar_per_hour/60, 2)
    # Time in minutes
    time_min = round(heist['time (hr)'] * 60, 3)
    # Distance between banks
    distance_between_banks_col = heist.apply(distance_between_banks, axis = 1)
    # Time between banks
    time_between_banks = (distance_between_banks_col/30) * 60
    # Distance to Helipad
    distance_to_helipad = pd.Series([distance(row['x_coordinate'], row['y_coordinate'], 0.0, 0.0) for index, row, in heist.iterrows()], index = heist.index)
    # Time to Helipad
    time_to_helipad = distance_to_helipad * 2
    # Total time needed
    total_time_needed = time_min + time_between_banks + time_to_helipad

    
    # Variable to account for total time allowed (24 Hours = 1440 Minutes)
    total_time_allowed = 1440
    # Variable to keep track of how much has been robbed so far
    total_robbed = 0
    # Output list that appends banks that have been robbed
    output = []

    # Transform the above variables from a Series to lists to pass to the algorithm
    bank_ids_lst = list(heist['id'])
    bank_money_lst = list(heist['money'])
    rate_per_minute_lst = list(rate_per_minute)
    time_to_rob_lst = list(time_min)
    time_banks_lst = list(time_between_banks)
    time_helipad_lst = list(time_to_helipad)
    total_time_lst = list(total_time_needed)

    
    # Create a main list to join all above lists
    heist_main_lst = list(zip(bank_ids_lst, bank_money_lst, rate_per_minute_lst, time_to_rob_lst, time_banks_lst, time_helipad_lst, total_time_lst))

    # Sort the Dataframe by rate per minute descendingly as we are using the greedy method
    heist_main_lst_sorted = sorted(heist_main_lst, key = lambda tup: tup[2], reverse = True)
    
    #Greedy Algorithm
    for i in range(len(heist_main_lst_sorted)):
        id, money,_, time_to_rob, time_between_banks,_, total_time_needed = heist_main_lst_sorted[i]
        if (total_time_needed < total_time_allowed):
            total_robbed += money
            output.append(id)
            total_time_allowed -= time_to_rob + time_between_banks

    
    return output
    
heist_algorithm(heist)

In [ ]:
'''
This is our final solution that we are submitting. It is the same as above but manipulating the DataFrame directly.
Within this solution we get a total of 6.4M with 0.13 mins left on the clock as per checker.
'''
def heist_algorithm(heist):
    heist['dollar_per_hr'] = heist['money']/heist['time (hr)']
    heist['rate_per_minute'] = round(heist['dollar_per_hr']/60, 2)
    heist['time (min)'] = round(heist['time (hr)'] * 60, 3)

    heist = heist.sort_values(by = 'rate_per_minute', ascending = False)
    heist.reset_index(inplace = True)
    heist['distance_between_banks'] = heist.apply(distance_between_banks, axis = 1)
    heist['time_between_banks'] = (heist['distance_between_banks']/30) * 60
    heist['distance_to_helipad'] = pd.Series([distance(row['x_coordinate'], row['y_coordinate'], 0.0, 0.0) for index, row, in heist.iterrows()], index = heist.index)
    heist['time_to_helipad'] = heist['distance_to_helipad'] * 2
    heist['total_time_needed'] = heist['time (min)'] + heist['time_between_banks'] + heist['time_to_helipad']

    bank_ids_lst = list(heist['id'])
    bank_money_lst = list(heist['money'])
    time_to_rob_lst = list(heist['time (min)'])
    time_banks_lst = list(heist['time_between_banks'])
    time_helipad_lst = list(heist['time_to_helipad'])
    total_time_lst = list(heist['total_time_needed'])
    heist_main_lst = list(zip(bank_ids_lst, bank_money_lst, time_to_rob_lst, time_banks_lst, time_helipad_lst, total_time_lst))

    total_time = 1440
    total_robbed = 0

    # List of Bank IDs
    output = []

    for i in range(len(heist_main_lst)):
        id, money, time_rob, time_banks,_, time_needed = heist_main_lst[i]
        if (time_needed < total_time):
            total_robbed += money
            output.append(id)
            total_time -= time_rob + time_banks

    #return len(output), total_robbed, total_time
    return output

heist_algorithm(heist)

In [9]:
import math


def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def check_solution(travel_list, df, speed=30.):
    """
    Check a bank robber algorithm solution
    
    input:
    ------
    travel_list: a list of integers
        Your solution to the bank robber problem
        Bank IDs are row indeces into the df passed
    df: pd.DataFrame
        The algorithm input formatted dataframe 
        (id,x_coordinate,y_coordinate,money,time (hr))
    speed: float
        The km/h speed of travel
    """
    t_remaining = 24.
    score = 0.
    assert len(travel_list) == len(set(travel_list)), (
        "Your travel list must have unique IDs!"
    )
    prev = travel_list[0]
    for e in travel_list:
        row = df.iloc[e]
        score += row['money']
        t_remaining -= row['time (hr)']
        dist = distance(
            row['x_coordinate'], row['y_coordinate'],
            df.iloc[prev]['x_coordinate'],
            df.iloc[prev]['y_coordinate'],
        )
        t_remaining -= dist / speed
        prev = e
    assert t_remaining >= 0, (
        f"Used more than 24h! Time left: {t_remaining}"
    )
    # still gotta get to (0, 0)
    dist = distance(row['x_coordinate'],row['y_coordinate'],0,0)
    final_t = t_remaining - (dist / speed)
    assert final_t >= 0, (
        f"Not enough time to get to helicopter!\n"
        f"Time left after last bank: {t_remaining}\n"
        f"Distance to (0,0) helipad: {dist}\n"
    )
    print(f"Time Remaining: {final_t}")
    return score

check_solution(heist_algorithm(heist), heist, speed =30.)

Time Remaining: 0.1693288549061656


6437500.0